In [ ]:
pip install openai xlutils xlwt

In [ ]:
import os
import openai
import xlrd
import xlwt
import random
from xlutils.copy import copy
from xlwt import Workbook

# define list of OpenAI API keys
api_keys = ["API_KEY"]
# randomly select an API key
api_key = random.choice(api_keys)

# set OpenAI API key
openai.api_key = api_key

# define OpenAI prompts
prompts = {
    'Study Design': 'Given the following abstract, what is the study design? please dont write sentence. just give me maximum 5 words. \n\nAbstract: ',
    'Study Population/Disease/Situation': 'Given the following abstract, what is the study population? dont write sentence. just give me words. \n\nAbstract: ',
    'Study Comparison/Prognostic Factor': 'Given the following abstract, what is the study comparison? dont write sentence. just give me words. \n\nAbstract: ',
    'Study Exposure/Intervention': 'Given the following abstract, what is the study exposure/intervention? dont write sentence. just give me words. \n\nAbstract: ',
    'Study Primary Outcome': 'Given the following abstract, what is the study primary outcome? dont write sentence. just give me words. \n\nAbstract: ',
     }

# set the folder path and input/output file paths
folder_path = '/content/text_abstracts'
input_path = '/content/results.xls'
output_path = '/content/results.xls'

# open the input file and create a copy to write the results
input_book = xlrd.open_workbook(input_path, formatting_info=True)
input_sheet = input_book.sheet_by_index(0)
output_book = copy(input_book)
output_sheet = output_book.get_sheet(0)

# get the number of rows in the input file
num_rows = input_sheet.nrows

# iterate over the files in the folder
for i, filename in enumerate(os.listdir(folder_path)):
    # check if the file is a text file and contains more than 300 words
    if filename.endswith('.txt'):
        with open(os.path.join(folder_path, filename), 'r') as file:
            text = file.read()
        if len(text.split()) > 300:
            # initialize a dictionary to store the results of each prompt
            results = {'File Name': filename}


            # iterate over each prompt and generate the response using OpenAI API
            for key, prompt in prompts.items():
                response = openai.Completion.create(
                    engine="text-davinci-002",
                    prompt=prompt + text,
                    max_tokens=600,
                    n=1,
                    stop=None,
                    temperature=0.5,
                )
                generated_text = response.choices[0].text.strip()
                results[key] = generated_text

            # write the results to the output file
            row = num_rows + i
            for col, key in enumerate(results.keys()):
                output_sheet.write(row, col, results[key])
            output_book.save(output_path)
